In [1]:
## Path for modules

import sys

import numpy as np
import xarray as xr

from netCDF4 import Dataset

sys.path.insert(0,"/mnt/meom/workdir/henelle/Notebooks/git/xscale")
import xscale
import xscale.spectral.fft as xfft

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy.ma as ma

import matplotlib.cm as mplcm
import matplotlib.colors as colors
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import date, datetime
from xhistogram.xarray import histogram

import pandas as pd

import seaborn as sns
sns.set(color_codes=True)

%matplotlib inline

In [2]:
def plot_surf_plus_histo(data1,data2,hist_data1_00,hist_data1_T0,hist_data2_00,hist_data2_T0,lon,lat,i,lonmin,lonmax,
                         latmin,latmax,vmin_data1,vmax_data1,vmin_data2,vmax_data2,cmap_data1,cmap_data2,
                         var_title_data1,var_title_data2,date,season):
    
    fig = plt.figure(figsize=(14,20))
    gs = gridspec.GridSpec(nrows=2, ncols=2, height_ratios=[2, 1.25]) 
    
    
    ax1 = fig.add_subplot(gs[0, 0],projection=ccrs.PlateCarree())#221
    ax2 = fig.add_subplot(gs[0, 1],projection=ccrs.PlateCarree())#222
    ax3 = fig.add_subplot(gs[1, 0])#223
    ax4 = fig.add_subplot(gs[1, 1])#224
    
    # Adjust subplots
    plt.subplots_adjust(hspace=0.0,wspace=0.1)
    
    norm_fld_data1 = colors.Normalize(vmin=vmin_data1, vmax=vmax_data1, clip=False)
    norm_fld_data2 = colors.Normalize(vmin=vmin_data2, vmax=vmax_data2, clip=False)
    
    pcolor1 = ax1.pcolormesh(lon,lat,data1,cmap=cmap_data1,vmin=vmin_data1,vmax=vmax_data1,norm = norm_fld_data1)
    pcolor2 = ax2.pcolormesh(lon,lat,data2,cmap=cmap_data2,vmin=vmin_data2,vmax=vmax_data2,norm = norm_fld_data2)
    
    # Add the Azores
    land = cfeature.GSHHSFeature(scale='intermediate',
                                 levels=[1],
                                 facecolor='dimgray')
    ax1.add_feature(land)
    ax2.add_feature(land)
    
    
    # Colorbar ax1
    cax1,kw1   = mpl.colorbar.make_axes(ax1,location='bottom',pad=0.05,shrink=0.95)
    out1       = fig.colorbar(pcolor1,cax=cax1,extend='both',**kw1)
    out1.set_label(var_title_data1,size=16)
    out1.ax.tick_params(labelsize=16)
    xticks_ax1 = np.linspace(vmin_data1,vmax_data1,num=5)
    out1.set_ticks(xticks_ax1)
    
    # Colorbar ax2
    cax2,kw2   = mpl.colorbar.make_axes(ax2,location='bottom',pad=0.05,shrink=0.95)
    out2       = fig.colorbar(pcolor2,cax=cax2,extend='max',**kw2)
    out2.set_label(var_title_data2,size=16)
    out2.ax.tick_params(labelsize=16)
    xticks_ax2 = np.linspace(vmin_data2,vmax_data2,num=5)
    out2.set_ticks(xticks_ax2)
    
    # Grid    
    gl1            = ax1.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,lw=1,color='gray',alpha=0.8, ls='--')
    gl1.xformatter = LONGITUDE_FORMATTER
    gl1.yformatter = LATITUDE_FORMATTER
    gl1.xlabel_style = {'size': 13, 'color': 'dimgray'}
    gl1.ylabel_style = {'size': 13, 'color': 'dimgray'}
    gl1.xlabels_top = False
    gl1.ylabels_right = False
    gl2            = ax2.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,lw=1,color='gray',alpha=0.8, ls='--')
    gl2.xformatter = LONGITUDE_FORMATTER
    gl2.yformatter = LATITUDE_FORMATTER
    gl2.xlabel_style = {'size': 13, 'color': 'dimgray'}
    gl2.ylabel_style = {'size': 13, 'color': 'dimgray'}
    gl2.xlabels_top = False
    gl2.ylabels_right = False
    gl2.ylabels_left = False
    
    # Adjusting axes
    ax1.set_xlim((-36, -26))
    ax1.set_ylim((25, 40))
    ax2.set_xlim((-36, -26))
    ax2.set_ylim((25, 40))
    
    
    # Chosen region
    ax1.plot([lonmin, lonmax], [latmin, latmin],color='black',linewidth=4)
    ax1.plot([lonmin, lonmax], [latmax, latmax],color='black',linewidth=4)
    ax1.plot([lonmin, lonmin], [latmin, latmax],color='black',linewidth=4)
    ax1.plot([lonmax, lonmax], [latmin, latmax],color='black',linewidth=4)
    ax2.plot([lonmin, lonmax], [latmin, latmin],color='black',linewidth=4)
    ax2.plot([lonmin, lonmax], [latmax, latmax],color='black',linewidth=4)
    ax2.plot([lonmin, lonmin], [latmin, latmax],color='black',linewidth=4)
    ax2.plot([lonmax, lonmax], [latmin, latmax],color='black',linewidth=4)
    
    
    # Adding histograms
    hist_data1_00.plot(ax=ax3,label='no tide',lw=3.0,color='lightseagreen')
    hist_data1_T0.plot(ax=ax3,label='tide',lw=3.0,linestyle='-.',color='darkred')
    ax3.set_xlabel('24h filtered '+var_title_data1,size=16)
    ax3.set_ylabel('')
    ax3.tick_params(labelsize=15)

    hist_data2_00.plot(ax=ax4,label='no tide',lw=3.0,color='lightseagreen')
    hist_data2_T0.plot(ax=ax4,label='tide',lw=3.0,linestyle='-.',color='darkred')
    ax4.set_xlabel('24h filtered '+var_title_data2,size=16)
    ax4.set_ylabel('')
    ax4.tick_params(labelsize=15)
    
    # Designing legend
    plt.draw()
    handles, labels = ax3.get_legend_handles_labels()
    fig.legend(handles, labels, loc='center',frameon=True,shadow=True,fontsize=16,bbox_to_anchor=(0.5, 0.07, 0.5, 0.5),
               markerscale=5.0,framealpha=0.8,edgecolor='black',facecolor='white')
    
    # Adding text box for defining region
    props   = dict(boxstyle='round', facecolor='orange', alpha=0.5)
    textstr = 'Region:\n \n '+str(np.abs(lonmin))+'$^o$W to '+str(np.abs(lonmax))+'$^o$W \n '+str(latmin)+'$^o$N to '+str(latmax)+'$^o$N'
    ax4.text(0.6,0.7,textstr,bbox=props,size=16,transform=ax4.transAxes)

    ax3.set_xlim((-0.5,0.5))
    ax4.set_xlim((0,0.5))
    
    
    # Figure titles
    ts = pd.to_datetime(str(date))
    d = ts.strftime('%d/%m/%y %H:%M')
    
    ax1.set_title(var_title_data1+' no tide; '+str(d),size=16, y=1.05)
    ax2.set_title(var_title_data2+' no tide; '+str(d),size=16, y=1.05)
    plt.suptitle(season,size=20,y=0.925)
    #plt.tight_layout()
    
    plt.savefig('./maps_region_PDF/log_Box+Hist_24h_ave_after_mean_file_BOX_'+str(lonmin)+'_'+str(lonmax)+'_'+str(latmin)+'_'
                +str(latmax)+'_'+season+'_'+'.png',dpi=100,bbox_inches='tight',pad_inches=0.1)
    #plt.clf()
    

In [3]:
## Dataset

data_dirT0 = '/mnt/meom/workdir/henelle/eNATL60/eNATL60-BLBT02-S/1h/ACO/'
data_dir00 = '/mnt/meom/workdir/henelle/eNATL60/eNATL60-BLB002-S/1h/ACO/'

In [4]:
## All files, JAS

# No tide
tfilename_strain00 = data_dir00 + 'eNATL60ACO-BLB002_y2009*.1h_strain10m.nc'
tfilename_curl00   = data_dir00 + 'eNATL60ACO-BLB002_y2009*.1h_curl10m.nc'

# Tide
tfilename_strainT0 = data_dirT0 + 'eNATL60ACO-BLBT02_y2009*.1h_strain10m.nc'
tfilename_curlT0   = data_dirT0 + 'eNATL60ACO-BLBT02_y2009*.1h_curl10m.nc'

In [5]:
fstrain00 = xr.open_mfdataset(tfilename_strain00,combine='by_coords')
strain00 = fstrain00['sostrainoverf'][:]
navlat = fstrain00['nav_lat']
navlon = fstrain00['nav_lon']

fstrainT0 = xr.open_mfdataset(tfilename_strainT0,combine='by_coords')
strainT0=fstrainT0['sostrainoverf'][:]

fcurl00 = xr.open_mfdataset(tfilename_curl00,combine='by_coords')
curl00=fcurl00['socurloverf'][:]

fcurlT0   = xr.open_mfdataset(tfilename_curlT0,combine='by_coords')
curlT0 = fcurlT0['socurloverf'][:]

In [6]:
print(strainT0)
print(navlon)

<xarray.DataArray 'sostrainoverf' (time_counter: 2208, y: 1191, x: 781)>
dask.array<concatenate, shape=(2208, 1191, 781), dtype=float32, chunksize=(24, 1191, 781), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2009-09-30T23:30:00
Dimensions without coordinates: y, x
Attributes:
    units:             -
    valid_min:         -1000.0
    valid_max:         1000.0
    long_name:         Relative_Vorticity (strain)
    short_name:        sostrain
    iweight:           1
    online_operation:  N/A
    axis:              TYX
    savelog10:         0.0
<xarray.DataArray 'nav_lon' (time_counter: 2208, y: 1191, x: 781)>
dask.array<concatenate, shape=(2208, 1191, 781), dtype=float32, chunksize=(24, 1191, 781), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2009-09-30T23:30:00
Dimensions without coordinates: y, x
Attributes:
    standard_name:  longitude
    long_name: 

In [7]:
T=2*np.pi/(1E-4)

In [8]:
wstrainT0 = strainT0.window
wstrainT0.set(n=48,dim='time_counter', cutoff=2*T)
strainT0_filt = wstrainT0.convolve()

In [9]:
wstrain00 = strain00.window
wstrain00.set(n=48,dim='time_counter', cutoff=2*T)
strain00_filt = wstrain00.convolve()

In [10]:
wcurlT0 = curlT0.window
wcurlT0.set(n=48,dim='time_counter', cutoff=2*T)
curlT0_filt = wcurlT0.convolve()

In [11]:
wcurl00 = curl00.window
wcurl00.set(n=48,dim='time_counter', cutoff=2*T)
curl00_filt = wcurl00.convolve()
curl00_filt

<xarray.DataArray 'socurloverf' (time_counter: 2208, y: 1191, x: 781)>
dask.array<mul, shape=(2208, 1191, 781), dtype=float32, chunksize=(24, 1191, 781), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2009-09-30T23:30:00
Dimensions without coordinates: y, x

In [12]:
## Selecting box

#box    = (-31.5,-27,30.6,36)
#box    = (-34.0,-31.5,25.2,28.8)
#box    = (-34.0,-31.5,34.2,37.8)
#box    = (-34.5,-32.0,34.8,38.4)
#box    = (-35.0,-32.5,35.4,39.0)

box=(-31,-28,33,36)
domain=(box[0]<navlon)*(navlon<box[1])*(box[2]<navlat)*(navlat<box[3])
where=np.where(domain)

lats=navlat[where]
lons=navlon[where]

ind = np.unravel_index(np.argmin(lats, axis=None), lats.shape)
jmin = where[0][ind[0]]
ind = np.unravel_index(np.argmax(lats, axis=None), lats.shape)
jmax = where[0][ind[0]]
ind = np.unravel_index(np.argmin(lons, axis=None), lons.shape)
imin = where[0][ind[1]]
ind = np.unravel_index(np.argmax(lons, axis=None), lons.shape)
imax = where[0][ind[1]]


ValueError: dimensions are too large; arrays and shapes with a total size greater than 'intp' are not supported.

jmin = where[0][0]
jmax = where[0][-1]
imin = where[1][0]
imax = where[1][-1]

In [ ]:
print('jmin = '+str(jmin)+'; jmax = '+str(jmax))
print('imin = '+str(imin)+'; imax = '+str(imax))

In [ ]:
strainT0_filtbox = strainT0_filt[:,jmin:jmax+1,imin:imax+1].stack(z=('x', 'y','time_counter'))
strain00_filtbox = strain00_filt[:,jmin:jmax+1,imin:imax+1].stack(z=('x', 'y','time_counter'))
curlT0_filtbox = curlT0_filt[:,jmin:jmax+1,imin:imax+1].stack(z=('x', 'y','time_counter'))
curl00_filtbox = curl00_filt[:,jmin:jmax+1,imin:imax+1].stack(z=('x', 'y','time_counter'))

In [ ]:
now_hist = datetime.now()
current_time_hist = now_hist.strftime("%H:%M:%S")
print("Current Time, before histograms =", current_time_hist)

La partie ci-dessous n'a pas encore été adapté.

### Histograms JAS

sbins = np.arange(0.0,0.5, 0.025)
cbins = np.arange(-0.5,0.5, 0.025)

hist_JAS_strain00_filtbox = histogram(strain00_filtbox,bins=[sbins])
hist_JAS_strainT0_filtbox = histogram(strainT0_filtbox,bins=[sbins])
hist_JAS_curl00_filtbox   = histogram(curl00_filtbox,bins=[cbins])
hist_JAS_curlT0_filtbox   = histogram(curlT0_filtbox,bins=[cbins])

#np.seterr(divide = 'ignore')
#with np.errstate(divide='ignore'):

log_hist_JAS_strain00 = np.log10(hist_JAS_strain00)
log_hist_JAS_strainT0 = np.log10(hist_JAS_strainT0)
log_hist_JAS_curl00   = np.log10(hist_JAS_curl00)
log_hist_JAS_curlT0   = np.log10(hist_JAS_curlT0)

### JAS

plot_surf_plus_histo(ds_JAS_curl00.socurloverf[45,:,:],ds_JAS_strain00.sostrainoverf[45,:,:],log_hist_JAS_curl00,
                     log_hist_JAS_curlT0,log_hist_JAS_strain00,log_hist_JAS_strainT0,navlon,navlat,45,-34.5,-32.0,34.8,38.4,vmin_data1=-0.5,vmax_data1=0.5,vmin_data2=0,vmax_data2=0.5,
                     cmap_data1='RdYlBu_r',cmap_data2='YlGn',var_title_data1='curl/f',var_title_data2='strain/f',
                     date=str(ds_JAS_strain00.time_counter.values[45]),season='JAS')

weights_curltidefilt = np.ones_like(curltidefiltbox)/float(len(curltidefiltbox))
weights_curlnotidefilt =  np.ones_like(curlnotidefiltbox)/float(len(curlnotidefiltbox))
weights_straintidefilt = np.ones_like(straintidefiltbox)/float(len(straintidefiltbox))
weights_strainnotidefilt = np.ones_like(strainnotidefiltbox)/float(len(strainnotidefiltbox))

fig = plt.figure(figsize=(18.0, 12.0))
axes1 = fig.add_subplot(1, 2, 1)

nstraintide, binsstraintide, patchs = axes1.hist(straintidefiltbox,500, alpha = 0.5,range=(0,1),color='r', weights=weights_straintidefilt,log=True)
nstrainnotide, binsstrainnotide, patchs = axes1.hist(strainnotidefiltbox,500, alpha = 0.5,range=(0,1),color='b', weights=weights_strainnotidefilt,log=True)

ncurltide,binscurltide,patchs = axes1.hist(curltidefiltbox,500, alpha = 0.5,range=(-1,1),color='r', weights=weights_curltidefilt,log=True)
ncurlnotide,binscurlnotide,patchs = axes1.hist(curlnotidefiltbox,500, alpha = 0.5,range=(-1,1),color='b', weights=weights_curlnotidefilt,log=True)

plt.clf()
fig = plt.figure(figsize=(18.0, 12.0))
axes1 = fig.add_subplot(1, 2, 1)

maxmax=np.max([np.max(nstraintide),np.max(nstrainnotide)])
axes1.plot(binsstraintide[0:-1],nstraintide/maxmax, color='b',linestyle='--',label='tide')
axes1.plot(binsstrainnotide[0:-1],nstrainnotide/maxmax, color='b',linestyle='-', label='no tide')
axes1.set_ylim(0.0001,1)
axes1.set_xlim(0,1)
plt.yscale('log')
plt.xticks([0,0.2,0.4,0.6,0.8,1])
plt.tick_params('both',labelsize=18)
#plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.xlabel('surface filtered strain/f',fontsize=22)
plt.legend(fontsize=18)
plt.grid(True,which='minor',axis='y', linestyle='--')
plt.grid(True,which='major',axis='both')

axes1 = fig.add_subplot(1, 2, 2)

maxmax=np.max([np.max(ncurltide),np.max(ncurlnotide)])
axes1.plot(binscurltide[0:-1],ncurltide/maxmax, color='b',linestyle='--',label='tide')
axes1.plot(binscurlnotide[0:-1],ncurlnotide/maxmax, color='b',linestyle='-', label='no tide')
axes1.set_xlim(-1,1)
axes1.set_ylim(0.0001,1)
plt.yscale('log')
plt.xticks([-1,-0.5,0,0.5,1])
plt.tick_params('both',labelsize=18)
#plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.xlabel('surface filtered curl/f',fontsize=22)
plt.legend(fontsize=18)
plt.grid(True,which='minor',axis='y', linestyle='--')
plt.grid(True,which='major',axis='both')

In [ ]:
now_end_plot = datetime.now()
current_time_end_plot = now_end_plot.strftime("%H:%M:%S")
print("Current Time, after plotting =", current_time_end_plot)